In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

import tensorflow as tf
print(tf.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

2025-04-29 01:19:10.386290: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-29 01:19:10.406479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745860750.424982   59485 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745860750.430342   59485 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745860750.443047   59485 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

2.19.0


In [2]:
with h5py.File("Data/Emotify_Annotated_Data.h5", "r") as hf:
    train_data = hf["train"][:]  # Load the train dataset
    label_data = hf["label"][:]  # Load the label dataset

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

(400, 256, 1292) float32
(400, 7) int8


In [3]:
train_data = train_data[..., np.newaxis]

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

(400, 256, 1292, 1) float32
(400, 7) int8


In [4]:
input_shape = train_data.shape[1], train_data.shape[2], train_data.shape[3]
print(input_shape)

(256, 1292, 1)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming train_data and label_data are already loaded
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data, label_data, test_size=0.2, random_state=42)

# Optionally, split the training set further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Print the shapes to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

In [6]:
model = load_model("checkpoint_epoch-59_acc-0.8700_loss-0.0035.keras")
model.summary()

I0000 00:00:1745860791.672386   59485 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1745860791.673902   59485 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/nigelchua0412/.local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 644, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 644, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 322, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 322, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 161, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 31, 161, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 31, 161, 384)   │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 31, 161, 384)   │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 31, 161, 512)   │     1,769,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 31, 161, 512)   │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 5,191,255 (19.80 MB)

 Trainable params: 67,591 (264.03 KB)

 Non-trainable params: 4,988,480 (19.03 MB)

 Optimizer params: 135,184 (528.07 KB)

In [7]:
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [8]:
results = model.evaluate(train_data, label_data, batch_size=8, verbose=1)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])
# Predict the labels for the validation set

I0000 00:00:1745860802.904282   59657 service.cc:152] XLA service 0x7ff9e0023b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745860802.904320   59657 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-29 01:20:02.930304: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745860803.076953   59657 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/50 ━━━━━━━━━━━━━━━━━━━━ 8:35 11s/step - accuracy: 0.7500 - loss: 0.0913

I0000 00:00:1745860812.637998   59657 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.7171 - loss: 0.0575
Validation Loss: 0.05518512427806854
Validation Accuracy: 0.7124999761581421


In [9]:
# Assuming 'model' is your trained model
y_pred = model.predict(train_data)

# Convert probabilities to binary labels (e.g., threshold at 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

2025-04-29 01:20:24.758251: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2025-04-29 01:20:25.829611: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_264', 4 bytes spill stores, 4 bytes spill loads

2025-04-29 01:20:31.885460: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.28 = (f32[32,64,126,644]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,126,644]{3,2,1,0} %bitcast.818, f32[64,64,3,3]{3,2,1,0} %bitcast.825, f32[64]{0} %bitcast.827), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_3_1/conv2d_1_2/convolution" source_file="/home/nigelchua0412/.local/lib/python3.12/site-package

13/13 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step 


In [11]:

label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']
# Generate the classification report
report = classification_report(label_data, y_pred_binary, target_names=label_names)
print(report)

# Generate the multilabel confusion matrix
mcm = multilabel_confusion_matrix(label_data, y_pred_binary)
print("Multilabel Confusion Matrix:")
print(mcm)

                   precision    recall  f1-score   support

           melody       1.00      0.92      0.96       103
     articulation       1.00      0.94      0.97        88
rhythm_complexity       1.00      0.89      0.94        19
 rhythm_stability       1.00      0.89      0.94        81
       dissonance       1.00      0.78      0.88        27
        atonality       1.00      0.94      0.97       208
             mode       1.00      0.93      0.96        67

        micro avg       1.00      0.92      0.96       593
        macro avg       1.00      0.90      0.95       593
     weighted avg       1.00      0.92      0.96       593
      samples avg       0.91      0.87      0.88       593

Multilabel Confusion Matrix:
[[[297   0]
  [  8  95]]

 [[312   0]
  [  5  83]]

 [[381   0]
  [  2  17]]

 [[319   0]
  [  9  72]]

 [[373   0]
  [  6  21]]

 [[192   0]
  [ 13 195]]

 [[333   0]
  [  5  62]]]


/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

# Plot the confusion matrix for each label
for i, cm in enumerate(mcm):
    print(f'Confusion Matrix for {label_names[i]}')
    print('Actual', cm)
    print("Predicted")
    # plt.figure(figsize=(6, 4))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    # plt.title(f'Confusion Matrix for {label_names[i]}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()


Confusion Matrix for melody
Actual [[297   0]
 [  8  95]]
Predicted
Confusion Matrix for articulation
Actual [[312   0]
 [  5  83]]
Predicted
Confusion Matrix for rhythm_complexity
Actual [[381   0]
 [  2  17]]
Predicted
Confusion Matrix for rhythm_stability
Actual [[319   0]
 [  9  72]]
Predicted
Confusion Matrix for dissonance
Actual [[373   0]
 [  6  21]]
Predicted
Confusion Matrix for atonality
Actual [[192   0]
 [ 13 195]]
Predicted
Confusion Matrix for mode
Actual [[333   0]
 [  5  62]]
Predicted


In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import ace_tools as tools
# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

metrics = []

for i, cm in enumerate(mcm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / cm.sum()
    metrics.append({
        'label': label_names[i],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    })

df_metrics = pd.DataFrame(metrics)
# Display metrics table
print(df_metrics)
# tools.display_dataframe_to_user(name="Label-wise Metrics", dataframe=df_metrics)

# # Plot all confusion matrices in a grid
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# for i, cm in enumerate(cms):
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['Pred Neg', 'Pred Pos'],
#                 yticklabels=['True Neg', 'True Pos'],
#                 ax=axes[i])
#     axes[i].set_title(label_names[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('Actual')

# plt.tight_layout()
plt.show()

               label  precision    recall  f1_score  accuracy
0             melody        1.0  0.922330  0.959596    0.9800
1       articulation        1.0  0.943182  0.970760    0.9875
2  rhythm_complexity        1.0  0.894737  0.944444    0.9950
3   rhythm_stability        1.0  0.888889  0.941176    0.9775
4         dissonance        1.0  0.777778  0.875000    0.9850
5          atonality        1.0  0.937500  0.967742    0.9675
6               mode        1.0  0.925373  0.961240    0.9875


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    preds = (model.predict(train_data) > t).astype(int)
    precision = precision_score(label_data, preds, average='macro')
    recall = recall_score(label_data, preds, average='macro')
    f1 = f1_score(label_data, preds, average='macro')
    accuracy = accuracy_score(label_data, preds)
    print(f"\nThreshold: {t}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")


NameError: name 'X_val' is not defined